In [1]:
# Description: In this version of the algorithm, the voxels are checked in specified scanning or sweeping directions (i.e. along the
# positive and negative x directions , positive and negative y directions and positive and negative z directions). Each voxel is 
# checked only for color consistency.

import os
import cv2
import math
import numpy as np 
from numpy import savetxt, genfromtxt

import silhouette_extract
import projection
import radiance_max_dist
import radiance_std


#Setting the path
data_dir = os.path.expanduser('~')
data_dir = data_dir + "/Downloads/DataSets/bird_data/images/"

#Dimensions of the initial canvas of voxels 

# Dimensions for Beethoven's head           
step_size = 0.25
xmin = -8
xmax = 5
ymin = -8
ymax = 8
zmin = -5
zmax = 16 #17.5

#Dimensions for Bunny

#Dimensions for Bird

#Dimensions for Pig



total_voxels=int((xmax-xmin)/step_size)*int((ymax-ymin)/step_size)*int((zmax-zmin)/step_size)

#Image dimensions
height = 768
width = 1024

#Initial canvas of voxels defined as a three dimentional array of true booleans
obj = np.ones((int((xmax-xmin)/step_size), int((ymax-ymin)/step_size), int((zmax-zmin)/step_size)), dtype = bool)


out_cnt = 0
bkgrnd_image_pixel_mark_situ = {}
color_image_pixel_mark_situ = {}
silhouettes = {}
camera_projections = {}
images = {}
camera_positions = {}

for file in os.listdir(data_dir):
    #Initializing the pixel marking matrix to account for voxels occlusion from some camera views
    bkgrnd_pixel_map = np.zeros((height, width), dtype = np.int8)
    color_pixel_map = np.zeros((height, width), dtype = np.int8)
    bkgrnd_image_pixel_mark_situ[file] = bkgrnd_pixel_map
    color_image_pixel_mark_situ[file] = color_pixel_map
    silhouettes[file] = silhouette_extract.get_mask(file)
    camera_projections[file] = projection.projectionMatrix(file)
    images[file] = cv2.imread(data_dir + file)
    camera_positions[file] = -1*np.matmul(np.linalg.inv(camera_projections[file][:,:3]),camera_projections[file][:,3])#The camera 
    #position is necessary to keep track of the cameras that are located behind the sweeping plane as it advances through the canvas
    #of voxels

#Method to check the voxel consistency using only the Lambertian color constraint
def color_consistency_check(i, j, k, camera_set):
    global color_image_pixel_mark_situ
    color_consist = True

    color_set = np.empty([len(images), 3])
    c = 0
    voxel_cord = np.array([[xmin + i*step_size], [ymin + j*step_size],[zmin + k*step_size], [1]])

    for file in camera_set:
        img = images[file]
        pix_cord = camera_projections[file].dot(voxel_cord)
        pix_cord = pix_cord / pix_cord[2,0]

        if pix_cord[1,0] >= height or pix_cord[0,0] < 0 or pix_cord[0,0] >= width or pix_cord[1,0] < 0:
            continue    
        
        if color_image_pixel_mark_situ[file][int(pix_cord[1,0]), int(pix_cord[0,0])] == 0:
            color_set[c] = img[int(pix_cord[1,0]), int(pix_cord[0,0])]
            c = c+1
    if c == 0:
        return True
    if c != len(images):
        #print("Color set before splitting: ", color_set)
        color_set = np.split(color_set, [c, len(images)], axis = 0)[0]
        #print("Color Set: ", color_set)
    color_consist, color = radiance_max_dist.consistency(color_set)
    #color_consist, color = radiance_std.consistency(color_set)
    
    if not color_consist:
        return False
    else:
        
        # area block. This idea is implemented to avoid the extraction and the inclusion in the 
        # calculation of the projected pixel's color from an image where the correspondent voxel cannot be seen
        # in the first place (i.e. the correspondent voxel is hidden by another voxel that was previously proven
        # to be consistent)
        voxel_cord1 = np.array([[xmin + i*step_size+step_size], [ymin + j*step_size+step_size],[zmin + k*step_size+step_size], [1]])
        for file in os.listdir(data_dir):
            pix_cord = camera_projections[file].dot(voxel_cord)
            pix_cord = pix_cord / pix_cord[2,0]
            if pix_cord[1,0] >= height or pix_cord[0,0] < 0 or pix_cord[0,0] >= width or pix_cord[1,0] < 0:
                continue 
            if color_image_pixel_mark_situ[file][int(pix_cord[1,0]), int(pix_cord[0,0])] == 0:
                pix_cord1 = camera_projections[file].dot(voxel_cord1)
                pix_cord1 = pix_cord1 / pix_cord1[2,0]
                pix_length = int(np.linalg.norm(pix_cord-pix_cord1))
        
                for m in range(int(pix_cord[1,0]-pix_length/2),int(pix_cord[1,0] + pix_length/2)):
                    for n in range(int(pix_cord[0,0]-pix_length/2),int(pix_cord[0,0] + pix_length/2)):
                        if ((m>=height or m<0) or (n>=width or n<0)):
                            continue
                        else:
                            color_image_pixel_mark_situ[file][m,n] = 100 #any other non-zero number can be placed here
        return True       

if __name__ == "__main__":
    voxel_count = 0
    eval_list = []
    # carving out the visual hull
    # begin plane sweeps

    #########################################_x_ positive_####################################
    voxel_count = 0
    x_pos_carve_cnt = 0
    for i in range(obj.shape[0]):
        # form the camera set
        threshold = xmin + i*step_size
        camera_set = [] 
        for key,values in  camera_positions.items():
            if(values[0] < threshold):
                camera_set.append(key)
        if len(camera_set) < 1:
            continue

        for j in range(obj.shape[1]):
            for k in range(obj.shape[2]):
                voxel_count +=1
                if voxel_count%50000==0:
                    print("x_pos sweeping processed "+ str(voxel_count/total_voxels*100) + "% of voxels")

                if obj[i,j,k] == 0:
                    continue
                color_consistency = color_consistency_check(i,j,k,camera_set)
                if (not color_consistency):
                    obj[i,j,k] = 0
                    x_pos_carve_cnt = x_pos_carve_cnt+1
    print("positive x sweep carved:",x_pos_carve_cnt)

    # #########################################_x_negative_####################################
    voxel_count = 0
    x_neg_carve_cnt = 0
    for i in range(obj.shape[0]-1, 0, -1):
        # form the camera set
        threshold = xmin + i*step_size
        camera_set = [] 
        for key,values in  camera_positions.items():
            if(values[0] > threshold):
                camera_set.append(key)
        if len(camera_set) < 1:
            continue

        for j in range(obj.shape[1]):
            for k in range(obj.shape[2]):
                voxel_count +=1
                if voxel_count%50000==0:
                    print("x_neg sweeping processed "+ str(voxel_count/total_voxels*100) + "% of voxels")

                if obj[i,j,k] == 0:
                    continue
                color_consistency = color_consistency_check(i,j,k,camera_set)
                if (not color_consistency):
                    obj[i,j,k] = 0
                    x_neg_carve_cnt = x_neg_carve_cnt+1
    print("negative x sweep carved:",x_neg_carve_cnt)

    #########################################_y_ positive_####################################
    voxel_count = 0
    y_pos_carve_cnt = 0
    for j in range(obj.shape[1]):
        # form the camera set
        threshold = ymin + j*step_size
        camera_set = [] 
        for key,values in  camera_positions.items():
            if(values[1] < threshold):
                camera_set.append(key)
        if len(camera_set) < 1:
            continue

        for i in range(obj.shape[0]):
            for k in range(obj.shape[2]):
                voxel_count +=1
                if voxel_count%50000==0:
                    print("y_pos sweeping processed "+ str(voxel_count/total_voxels*100) + "% of voxels")

                if obj[i,j,k] == 0:
                    continue
                color_consistency = color_consistency_check(i,j,k,camera_set)
                if (not color_consistency):
                    obj[i,j,k] = 0
                    y_pos_carve_cnt = y_pos_carve_cnt+1
    print("positive y sweep carved:",y_pos_carve_cnt)

    # #########################################_y_negative_####################################
    voxel_count = 0
    y_neg_carve_cnt = 0
    for j in range(obj.shape[1]-1, 0, -1):
        # form the camera set
        threshold = ymin + j*step_size
        camera_set = [] 
        for key,values in  camera_positions.items():
            if(values[1] > threshold):
                camera_set.append(key)
        if len(camera_set) < 1:
            continue

        for i in range(obj.shape[0]):
            for k in range(obj.shape[2]):
                voxel_count +=1
                if voxel_count%50000==0:
                    print("y_neg sweeping processed "+ str(voxel_count/total_voxels*100) + "% of voxels")

                if obj[i,j,k] == 0:
                    continue
                color_consistency = color_consistency_check(i,j,k,camera_set)
                if (not color_consistency):
                    obj[i,j,k] = 0
                    y_neg_carve_cnt = y_neg_carve_cnt+1
    print("negative y sweep carved:",y_neg_carve_cnt)

    #########################################_z_ positive_####################################
    voxel_count = 0
    z_pos_carve_cnt = 0
    for k in range(obj.shape[2]):
        # form the camera set
        threshold = zmin + k*step_size
        camera_set = [] 
        for key,values in  camera_positions.items():
            if(values[2] < threshold):
                camera_set.append(key)
        if len(camera_set) < 1:
            continue

        for i in range(obj.shape[0]):
            for j in range(obj.shape[1]):
                voxel_count +=1
                if voxel_count%50000==0:
                    print("z_pos sweeping processed "+ str(voxel_count/total_voxels*100) + "% of voxels")

                if obj[i,j,k] == 0:
                    continue
                color_consistency = color_consistency_check(i,j,k,camera_set)
                if (not color_consistency):
                    obj[i,j,k] = 0
                    z_pos_carve_cnt = z_pos_carve_cnt+1
    print("positive z sweep carved:",z_pos_carve_cnt)

    # #########################################_z_negative_####################################
    voxel_count = 0
    z_neg_carve_cnt = 0
    for k in range(obj.shape[2]-1, 0, -1):
        # form the camera set
        threshold = zmin + k*step_size
        camera_set = [] 
        for key,values in  camera_positions.items():
            if(values[2] > threshold):
                camera_set.append(key)
        if len(camera_set) < 1:
            continue

        for i in range(obj.shape[0]):
            for j in range(obj.shape[1]):
                voxel_count +=1
                if voxel_count%50000==0:
                    print("z_neg sweeping processed "+ str(voxel_count/total_voxels*100) + "% of voxels")

                if obj[i,j,k] == 0:
                    continue
                color_consistency = color_consistency_check(i,j,k,camera_set)
                if (not color_consistency):
                    obj[i,j,k] = 0
                    z_neg_carve_cnt = z_neg_carve_cnt+1
    print("negative z sweep carved:",z_neg_carve_cnt)

    #Saving the output 3D shape that was carved from the initial canvas of voxels as a .txt file
    print("Object size before carving: ",obj.size,"Object size after carving: ", obj.size-x_pos_carve_cnt-x_neg_carve_cnt-y_pos_carve_cnt-y_neg_carve_cnt-z_pos_carve_cnt-z_neg_carve_cnt,"Out count: ", out_cnt)
    final_file = open("./multiplane_color_only_visual_hull_result.txt", "w")
    head_line = "{} {} {} {} {} {} {} {}\n" # step_size, xmin, xmax, ymin, ymax, zmin, zmax, is_colored version
    voxel_elem = "{} "
    final_file.write(head_line.format(step_size, xmin, xmax, ymin, ymax, zmin, zmax, 0))
            
    for i in range(obj.shape[0]):
        for j in range(obj.shape[1]):
            for k in range(obj.shape[2]):
                final_file.write(voxel_elem.format(int(obj[i,j,k])))
                eval_list.append(int(obj[i,j,k]))  

    final_file.close()
    eval_array = np.array(eval_list)
    savetxt('multiplane_color_only.csv',eval_array,delimiter=',')                                   
# end of the visual hull build


x_pos sweeping processed 17.885760073260073% of voxels
x_pos sweeping processed 35.771520146520146% of voxels
x_pos sweeping processed 53.657280219780226% of voxels
x_pos sweeping processed 71.54304029304029% of voxels
x_pos sweeping processed 89.42880036630036% of voxels
positive x sweep carved: 11230
x_neg sweeping processed 17.885760073260073% of voxels
x_neg sweeping processed 35.771520146520146% of voxels
x_neg sweeping processed 53.657280219780226% of voxels
x_neg sweeping processed 71.54304029304029% of voxels
x_neg sweeping processed 89.42880036630036% of voxels
negative x sweep carved: 419
y_pos sweeping processed 17.885760073260073% of voxels
y_pos sweeping processed 35.771520146520146% of voxels
y_pos sweeping processed 53.657280219780226% of voxels
y_pos sweeping processed 71.54304029304029% of voxels
y_pos sweeping processed 89.42880036630036% of voxels
positive y sweep carved: 0
y_neg sweeping processed 17.885760073260073% of voxels
y_neg sweeping processed 35.77152014652